In [12]:
!pip install --upgrade tables
!pip install eli5
!pip install xgboost

Requirement already up-to-date: tables in /usr/local/lib/python3.6/dist-packages (3.6.1)


In [0]:
import pandas as pd
import numpy as np

from sklearn.dummy import DummyRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

import xgboost as xgb

from sklearn.metrics import mean_absolute_error as mae
from sklearn.model_selection import cross_val_score, KFold

import eli5
from eli5.sklearn import PermutationImportance

In [14]:
cd "/content/drive/My Drive/Colab Notebooks/matrix_transformacja2"

/content/drive/My Drive/Colab Notebooks/matrix_transformacja2


In [15]:
df = pd.read_hdf("data/car.h5")
df.shape

(106494, 155)

In [0]:
SUFFIX_CAT = '__cat'

for feat in df.columns:
  if  isinstance(df[feat][0], list) == False and SUFFIX_CAT not in feat:
    df[feat + SUFFIX_CAT] = df[feat].factorize()[0]

In [0]:
feats_cat = [x for x in df.columns if SUFFIX_CAT in x]
feats_cat = [x for x in feats_cat if 'price' not in x]

In [8]:
df.columns.values.size

309

In [9]:
len(feats_cat)

151

In [0]:
def run_model(feats,model):  
  X = df[feats].values
  y = df['price_value'].values
  
  scores = cross_val_score(model,X, y, cv=3, scoring='neg_mean_absolute_error')

  return np.mean(scores), np.std(scores)

## decission tree

In [24]:
model = DecisionTreeRegressor(max_depth=16)

run_model(feats=feats_cat,model=model)

(-15424.94822170682, 783.0797610295032)

## to teraz random forest

In [20]:
model = RandomForestRegressor(n_estimators=64, max_depth=16, random_state=1)

run_model(feats=feats_cat,model=model)

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py", line 2882, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-20-011d19a99f38>", line 3, in <module>
    run_model(feats=feats_cat,model=model)
  File "<ipython-input-18-0d37866d6e46>", line 5, in run_model
    scores = cross_val_score(model,X, y, cv=3, scoring='neg_mean_absolute_error')
  File "/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py", line 390, in cross_val_score
    error_score=error_score)
  File "/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py", line 236, in cross_validate
    for train, test in cv.split(X, y, groups))
  File "/usr/local/lib/python3.6/dist-packages/joblib/parallel.py", line 1007, in __call__
    while self.dispatch_one_batch(iterator):
  File "/usr/local/lib/python3.6/dist-packages/joblib/parallel.py", line 835, in dispatch_one_batch
    self._d

KeyboardInterrupt: ignored

## xgboost

In [22]:
xgb_params = dict(n_estimators=64, max_depth=16, learning_rate=0.1, seed=128)

run_model(feats=feats_cat,model=xgb.XGBRegressor(**xgb_params))

[20:14:24] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[20:16:22] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[20:18:22] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-10317.20005821073, 203.09693604418428)

In [32]:
df['param_moc'].unique()

array(['90 KM', '115 KM', '262 KM', '110 KM', '310 KM', '105 KM',
       '140 KM', '175 KM', '125 KM', '185 KM', '190 KM', '440 KM',
       '141 KM', '200 KM', '224 KM', '75 KM', '99 KM', '184 KM', '109 KM',
       '233 KM', '116 KM', '68 KM', '286 KM', '126 KM', '160 KM',
       '135 KM', '120 KM', '272 KM', None, '150 KM', '180 KM', '136 KM',
       '102 KM', '131 KM', '218 KM', '245 KM', '170 KM', '112 KM',
       '250 KM', '252 KM', '73 KM', '100 KM', '313 KM', '101 KM',
       '285 KM', '70 KM', '383 KM', '174 KM', '277 KM', '132 KM',
       '130 KM', '215 KM', '60 KM', '330 KM', '163 KM', '177 KM', '98 KM',
       '78 KM', '189 KM', '156 KM', '143 KM', '69 KM', '113 KM', '65 KM',
       '122 KM', '82 KM', '251 KM', '95 KM', '197 KM', '235 KM', '238 KM',
       '171 KM', '381 KM', '400 KM', '178 KM', '80 KM', '165 KM', '85 KM',
       '258 KM', '142 KM', '204 KM', '124 KM', '55 KM', '144 KM',
       '231 KM', '248 KM', '152 KM', '181 KM', '210 KM', '340 KM',
       '129 KM', '147 

In [0]:
df['param_rok-produkcji'] = df['param_rok-produkcji'].map(lambda x: -1 if str(x) == 'None' else x)

In [36]:
df['param_rok-produkcji'] = df['param_rok-produkcji'].map(lambda x: int(x))
df['param_rok-produkcji'].unique()

array([2018, 2011, 2015, 2009, 2017, 2012, 2013, 2007, 2001, 2016, 2006,
       2008, 2004, 1999, 2000, 2010, 2005, 2002, 1998, 2014, 2003, 1982,
       1995, 1997, 1992, 1993, 1994, 1996, 1989, 1988, 1967, 1987, 1970,
       1959, 1990, 1991, 1974,   -1, 1975, 1973, 1953, 1985, 1984, 1986,
       1981, 1979, 1960, 1983, 1978, 1964, 1980, 1972, 1969, 1956, 1966,
       1977, 1962, 1965, 1971, 1963, 1961, 1952, 1949, 1976, 1937, 1968,
       1958, 1955, 1933, 1929, 1957, 1944, 1954, 1932, 1936, 1947, 1948])

In [38]:
df['param_moc_new'] = df['param_moc'].map(lambda x: -1 if str(x) == 'None' else int(str(x).replace(' ', '').replace('KM','')) )
df['param_moc_new'].unique()

array([    90,    115,    262,    110,    310,    105,    140,    175,
          125,    185,    190,    440,    141,    200,    224,     75,
           99,    184,    109,    233,    116,     68,    286,    126,
          160,    135,    120,    272,     -1,    150,    180,    136,
          102,    131,    218,    245,    170,    112,    250,    252,
           73,    100,    313,    101,    285,     70,    383,    174,
          277,    132,    130,    215,     60,    330,    163,    177,
           98,     78,    189,    156,    143,     69,    113,     65,
          122,     82,    251,     95,    197,    235,    238,    171,
          381,    400,    178,     80,    165,     85,    258,    142,
          204,    124,     55,    144,    231,    248,    152,    181,
          210,    340,    129,    147,     50,     54,    290,    306,
          193,     77,    164,     96,    194,    111,    166,    206,
          118,    360,    211,    271,    455,    280,    106,    114,
      

In [42]:
df['param_pojemność-skokowa'] = df['param_pojemność-skokowa'].map(lambda x: -1 if str(x) == 'None' else int(str(x).replace(' ', '').replace('cm3','')))
df['param_pojemność-skokowa'].unique()

array([   898,   1560,   3000, ...,   5992,   1966, 142280])

In [0]:
feats = ['param_rok-produkcji','param_napęd__cat','param_stan__cat','param_faktura-vat__cat','param_moc_new','param_skrzynia-biegów__cat','param_marka-pojazdu__cat','param_pojemność-skokowa','feature_kamera-cofania__cat','param_wersja__cat','feature_łopatki-zmiany-biegów__cat','param_model-pojazdu__cat','param_kod-silnika__cat','feature_światła-led__cat','feature_bluetooth__cat','feature_regulowane-zawieszenie__cat','param_typ__cat','feature_asystent-pasa-ruchu__cat','feature_system-start-stop__cat','param_bezwypadkowy__cat']

In [44]:
xgb_params = dict(n_estimators=64, max_depth=16, learning_rate=0.1, seed=128)

run_model(feats=feats,model=xgb.XGBRegressor(**xgb_params))

[20:49:39] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[20:50:06] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[20:50:31] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-7863.786713939494, 89.55782585639136)

In [0]:
!git status

On branch master
Your branch is up to date with 'origin/master'.

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git checkout -- <file>..." to discard changes in working directory)

	modified:   matrixDay1.ipynb
	modified:   matrixDay2.ipynb
	modified:   matrixDay3.ipynb

Untracked files:
  (use "git add <file>..." to include in what will be committed)

	matrixDay4.ipynb

no changes added to commit (use "git add" and/or "git commit -a")


In [0]:
!git add 